In [ ]:
!pip install gunpowder

In [ ]:
pwd


In [ ]:
!echo $CONDA_DEFAULT_ENV

In [ ]:
!pip install gunpowder
!pip install zarr
!pip install matplotlib

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import zarr
from skimage import data
from skimage import filters
from scipy.ndimage import zoom

# make sure we all see the same
np.random.seed(19623)
random.seed(19623)

# open a sample image (channels first)
#raw_data2 = data.astronaut().transpose(2, 0, 1)

import skimage.io as sk

raw_data = sk.imread("sandbox/celegans/01/t000.tif", plugin="tifffile")
raw_data = zoom(np.random.rand(35, 10, 10), zoom=(1, 50, 70), order=0)

ModuleNotFoundError: No module named 'zarr'

In [ ]:
raw_data.shape

In [ ]:
plt.imshow(raw_data[20], cmap="magma")

In [ ]:
imstack1.shape

In [ ]:
raw_data.shape

In [ ]:

# create some dummy "ground-truth" to train on
gt_data = filters.gaussian(raw_data[0], sigma=3.0) > 0.75
gt_data = gt_data[np.newaxis,:].astype(np.float32)

# store image in zarr container
f = zarr.open('sample_data.zarr', 'w')
f['raw'] = raw_data
f['raw'].attrs['resolution'] = (1, 1, 1)
f['ground_truth'] = gt_data
f['ground_truth'].attrs['resolution'] = (1, 1, 1)

# helper function to show image(s), channels first
def imshow(raw, ground_truth=None, prediction=None):
  rows = 1
  if ground_truth is not None:
    rows += 1
  if prediction is not None:
    rows += 1
  cols = raw.shape[0] if len(raw.shape) > 3 else 1
  fig, axes = plt.subplots(rows, cols, figsize=(10, 4), sharex=True, sharey=True, squeeze=False)
  if len(raw.shape) == 3:
    axes[0][0].imshow(raw.transpose(1, 2, 0))
  else:
    for i, im in enumerate(raw):
      axes[0][i].imshow(im.transpose(1, 2, 0))
  row = 1
  if ground_truth is not None:
    if len(ground_truth.shape) == 3:
      axes[row][0].imshow(ground_truth[0])
    else:
      for i, gt in enumerate(ground_truth):
        axes[row][i].imshow(gt[0])
    row += 1
  if prediction is not None:
    if len(prediction.shape) == 3:
      axes[row][0].imshow(prediction[0])
    else:
      for i, gt in enumerate(prediction):
        axes[row][i].imshow(gt[0])
  plt.show()

In [ ]:
n = 19
imshow(zarr.open('sample_data.zarr')['raw'][n:n+1])

In [ ]:
import gunpowder as gp

# declare arrays to use in the pipeline
raw = gp.ArrayKey('RAW')


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# create "pipeline" consisting only of a data source
source = gp.ZarrSource(
    'sample_data.zarr',  # the zarr container
    {raw: 'raw'},  # which dataset to associate to the array key
    {raw: gp.ArraySpec(interpolatable=True)}  # meta-information
)
pipeline = source

In [ ]:
pipeline

In [ ]:
# formulate a request for "raw"
request = gp.BatchRequest()
request[raw] = gp.Roi((0, 0, 0), (4, 127, 176))

In [ ]:
raw_data.shape

In [ ]:
request[raw]

In [ ]:
gp.Roi?

In [ ]:
batch

In [ ]:
# build the pipeline...
with gp.build(pipeline):

  # ...and request a batch
  batch = pipeline.request_batch(request)

# show the content of the batch
print(f"batch returned: {batch}")
imshow(batch[raw].data)

In [ ]:
random_location = gp.RandomLocation()
pipeline = source + random_location

print(pipeline)


In [ ]:
with gp.build(pipeline):
    batch = pipeline.request_batch(request)

imshow(batch[raw].data)

### GEOMETRICAL AUGMENTATION

### Simple Augmentation

In [ ]:
simple_augment = gp.SimpleAugment()
pipeline = source + random_location + simple_augment

with gp.build(pipeline):
    batch = pipeline.request_batch(request)

#imshow(batch[raw].data)

In [ ]:
import math

elastic_augment = gp.ElasticAugment(
    [4,40,40],
    [0,2,2],
    [0,math.pi/2.0],
    prob_slip=0.05,
    prob_shift=0.05,
    max_misalign=25)

pipeline = source + random_location + simple_augment + elastic_augment

with gp.build(pipeline):
    batch = pipeline.request_batch(request)

#imshow(batch[raw].data)


#### Simple Augmentation

In [ ]:
#gp.SimpleAugment(transpose_only=[1, 2])

In [ ]:
# scale and shift the intensity of the raw array

In [ ]:
Intensity_augment=gp.IntensityAugment(
                    raw,
                    scale_min=0.9,
                    scale_max=1.1,
                    shift_min=-0.1,
                    shift_max=0.1,
                    z_section_wise=True) 

In [ ]:
Augmentations=
# chose a random source (i.e., sample) from the above
Augmentations=gp.RandomProvider() +

# elastically deform the batch
gp.ElasticAugment(
    [4,40,40],
    [0,2,2],
    [0,math.pi/2.0],
    prob_slip=0.05,
    prob_shift=0.05,
    max_misalign=25) +

# apply transpose and mirror augmentations
gp.SimpleAugment(transpose_only=[1, 2]) +

# scale and shift the intensity of the raw array
gp.IntensityAugment(
    raw,
    scale_min=0.9,
    scale_max=1.1,
    shift_min=-0.1,
    shift_max=0.1,
    z_section_wise=True) +

In [ ]:
gp.IntensityAugment?